In [ ]:
'''
%%writefile gap.py
start = time.time()
end = time.time()
print(end-start)
'''
import tensorflow as tf
import tensorflow_probability as tfp
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os as os
import sys
from pathlib import Path

mhsdir = Path(os.getcwd()).parent
sys.path.append(os.path.join(mhsdir, 'src'))

import ns
import gap

datarawdir = os.path.join(mhsdir, 'data\\raw\\YC')
dataprepdir = os.path.join(mhsdir, 'data\\preprocessed')
dataprepnsdir = os.path.join(mhsdir, 'data\\preprocessed\\ns')

tempdatarawdir = os.path.join(mhsdir, 'temp\\data\\raw\\YC')
tempdataprepdir = os.path.join(mhsdir, 'temp\\data\\preprocessed')
tempdataprepnsdir = os.path.join(mhsdir, 'temp\\data\\preprocessed\\ns')

tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers

# GAP

In [ ]:
restriction = tfk.constraints.NonNeg()
BATCH_SIZE = 32
dataset = gap.make_dataset(dataprepnsdir, eps = 0.00001).batch(BATCH_SIZE)
model = gap.GAPNSModel(event_shape = 5, gwide = 32, gdeep = 5, dwide = 32, ddeep = 5, penalty = 0.01, kernel_reg = gap.custom_reg)
model.fit(dataset, epochs = 1, batch = BATCH_SIZE)

In [ ]:
x = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 4.0,5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 
              84.0, 96.0, 108.0, 120.0, 144.0, 180.0, 240.0, 300.0, 360.0, 480.0, 600.0])  
x, y, _ = gap.generate_ycs(model)
plt.plot(x,y)

# Old implementation

In [ ]:
%%writefile gap.py
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)
cross_entropy = tfk.losses.BinaryCrossentropy(from_logits=True)
BATCH_SIZE = 16
noise_dim = 5
gauss_params = [0.0, 1.0]
generator_inputs = np.array([gauss_params] * BATCH_SIZE)

class StrictlyPositive(tfk.constraints.Constraint):
    def __init__(self, eps = 0.00001):
        self.eps = tf.constant(eps)
    def __call__(self, w):
        con = tfk.constraints.NonNeg()
        return self.eps + con(w)
    
restriction = tfk.constraints.NonNeg()


def make_dataset(srcdir, eps = 0.00001):
    dataset = None
    for filename in os.listdir(srcdir):
        df = pd.read_csv(os.path.join(srcdir,filename), usecols = ['Thau','Alpha0','Alpha1','Alpha2'])
        lx = df['Thau'].notnull()
        m = df[lx][['Thau','Alpha0','Alpha1','Alpha2']].to_numpy() + eps
        if dataset is None:
            dataset = m
        else:
            dataset = np.concatenate([dataset,m], axis = 0)
    return tf.data.Dataset.from_tensor_slices(dataset).shuffle(1000000)



generator = make_generator_model(event_shape = noise_dim, alpha = 0.3, wide = 32, deep = 4, weight_restriction = restriction)
discriminator = make_discriminator_model(wide = 32, deep = 4)

@tf.function
def train_step(nsv):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_nsv = generator(generator_inputs, training=True)
        
        real_output = discriminator(nsv, training=True)
        fake_output = discriminator(generated_nsv, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
def train(dataset, epochs):
    for epoch in range(epochs):
        print('Epoch - {}'.format(epoch))
        for ns_batch in dataset:
            train_step(ns_batch)

In [ ]:
x = np.array([0.25, 0.5, 1.0, 2.0, 3.0, 4.0,5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 24.0, 36.0, 48.0, 60.0, 72.0, 
              84.0, 96.0, 108.0, 120.0, 144.0, 180.0, 240.0, 300.0, 360.0, 480.0, 600.0])  
seed = np.array([gauss_params])
nsl = ns.NelsonSiegelLayer()
nsp = None
for _ in range(1000):
    nsp = generator(seed,training = False)
    nsp = tf.squeeze(nsp)
    if nsp[0] > 10.0 and nsp[0] < 120.0:
        break
nsl.assignValues(nsp)
y = nsl(x)
plt.plot(x,y)
print(nsl.weights)

In [ ]:
generator.save("generator")

In [ ]:
reconstructed_model = tfk.models.load_model("generator")